# Projekt Code Induktive Erwärmung

In [1]:
from ngsolve import *
from netgen.geom2d import unit_square
from ngsolve.webgui import Draw
from netgen.csg import *
from netgen.geom2d import SplineGeometry
import math
import numpy as np


### Konstanten

In [ ]:
omega = 2* pi *50   # Frequenz 50 Hz
mu0 = 4* pi *1e-7   # Magnetische Feldkonstante

sigmaCu = 56e6      # S/m (Kupfer)
sigmaCore = 56e6    # S/m (Kupfer)

sigma = CoefficientFunction ([0 , sigmaCu , sigmaCore ])
mur = CoefficientFunction ([1 ,1 ,1])
nu = CoefficientFunction (1/ mu0 *1/ mur ) # Permeabilität des Materials

ic = 10             # Strom in den Windungen (A)

# Mesh Generation
l_air = 0.1         # Grosse der dargestellten Luft um denn Versuchsaufbau
r_wire = 0.001      # Drahtdurchmesser in Metern
nx_wire = 10        # Anzahl Kabel in X richtung
ny_wire = 3         # Anzahl Kabel in Y richtung
dxdy_wire = 0.5*r_wire

rA_core = 0.01      # Core Höhe
rI_core = 0.008
l_core = 0.08       # Core Länge
epsZero = 1e-8

### Mesh generation

In [17]:
geo = SplineGeometry()
geo.AddRectangle(p1=(-l_air/2,epsZero),
                 p2=(l_air/2,l_air/2),
                 bcs=["rotsym","outer","outer","outer"],
                 leftdomain=1,
                 rightdomain=0)
pts = []
for i in range(nx_wire):
    for j in range(ny_wire):
        pts.append([
            i*(2*r_wire+dxdy_wire), 
            rA_core+dxdy_wire+r_wire+j*(2*r_wire+dxdy_wire)
        ])
        geo.AddCircle(
            c=(i*(2*r_wire+dxdy_wire), 
               rA_core+dxdy_wire+r_wire+j*(2*r_wire+dxdy_wire)),
            r=r_wire, bc="inner",
            leftdomain=2,
            rightdomain=1)
pts = np.array(pts)
geo.AddRectangle(p1=(-l_core/2,rI_core),
                 p2=(l_core/2,rA_core),
                 bc="inner",
                 leftdomain=3,
                 rightdomain=1)
geo.SetMaterial (1, "air")
geo.SetMaterial (2, "copper")
geo.SetMaterial (3, "core")
geo.SetDomainMaxH(3,(rA_core-rI_core)/2)

mesh = Mesh(geo.GenerateMesh(maxh=0.0025))

# Visualisierung
# Draw(mesh)

## Aufgabe 1
1) Wie lautet die schwache Gleichung für die partielle Differentialgleichung (4)? Verifizieren Sie diese am Listing 2.
2) Berechnen Sie die Lösung und stellen Sie diese graphisch dar. Nutze Sie für die
Berechnung die Formel (5).
3) Berechnen Sie die induktive Wärmequelle q mit Hilfe von Formel (7).

#### 1 Schwache Gleichung der partiellen Differtialgleichung

Die Differentialgleichung lautet:
$$
\sigma_j \omega A_\varphi - \frac{\partial}{\partial r} \left( \mu^{-1} r \frac{\partial (r A_\varphi)}{\partial r} \right) + \frac{\partial}{\partial z} \left( \mu^{-1} \frac{\partial A_\varphi}{\partial z} \right) = j_\varphi
$$

Die schwache Formulierung ergibt sich, indem man beide Seiten mit einer Testfunktion  $v$ multipliziert und über das Volumen  $\Omega$ integriert:

$$
\int_\Omega \left( \sigma_j \omega A_\varphi v - \frac{\partial}{\partial r} \left( \mu^{-1} r \frac{\partial (r A_\varphi)}{\partial r} \right) v + \frac{\partial}{\partial z} \left( \mu^{-1} \frac{\partial A_\varphi}{\partial z} \right) v \right) \, d\Omega = \int_\Omega j_\varphi v \, d\Omega
$$

Durch partielle Integration erhalten wir:

$$
\int_\Omega - \frac{\partial}{\partial r} \left( \mu^{-1} r \frac{\partial (r A_\varphi)}{\partial r} \right) v \, d\Omega = \int_\Omega \mu^{-1} r \frac{\partial (r A_\varphi)}{\partial r} \frac{\partial v}{\partial r} \, d\Omega
$$

$$
\int_\Omega - \frac{\partial}{\partial z} \left( \mu^{-1} \frac{\partial A_\varphi}{\partial z} \right) v \, d\Omega = \int_\Omega \mu^{-1} \frac{\partial A_\varphi}{\partial z} \frac{\partial v}{\partial z} \, d\Omega
$$

Die schwache Formulierung der Differentialgleichung lautet:

$$
a(A_\varphi, v) = f(v)
$$

wobei die Bilinearform  $a(A_\varphi, v)$  durch die folgenden Terme definiert ist:

$$
a(A_\varphi, v) = \int_\Omega \sigma_j \omega A_\varphi v \, d\Omega + \int_\Omega \mu^{-1} r \frac{\partial (r A_\varphi)}{\partial r} \frac{\partial v}{\partial r} \, d\Omega + \int_\Omega \mu^{-1} \frac{\partial A_\varphi}{\partial z} \frac{\partial v}{\partial z} \, d\Omega
$$

Der Lastvektor  $f(v)$  ist gegeben durch:

$$
f(v) = \int_\Omega j_\varphi v \, d\Omega
$$


In [20]:
### Listing 1 : Definition FE-Raum für Wirbelstromproblem
V = H1(mesh,order=3, complex = True, dirichlet='rotsym')
u,v = V.TnT()
gfu = GridFunction(V)

In [ ]:
### Listing 2 : Definition Wirbelstrom Problem

#Wechsel in Zylinderkoordinaten (er wird nur ein Queerschnitt dargestellt (phi = 0))
r = y
z = x
uz, ur = grad(u)
vz, vr = grad(v) 

a = BilinearForm(V) # Inegral über Omega
a += (nu*(1/r*u+ur)*vr + nu*uz*vz)*dx
a += 1j*omega*sigma*u*v*dx(definedon=mesh.Materials('copper|core'))

Jimp = CoefficientFunction([0,ic/(r_wire**2*pi),0])

f = LinearForm(V)
f += Jimp*v*dx(definedon=mesh.Materials('copper'))

a.Assemble()
f.Assemble()

gfu.vec.data = a.mat.Inverse(V.FreeDofs()) * f.vec

Draw(gfu)


WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {'Complex': {'phase': 0.0, 's…

BaseWebGuiScene

#### 2 Induktive Wärmequelle
Die Formel (7) für die induktive Wärmequelle lautet:

$$
q = \frac{1}{2} | \mathbf{J} \cdot \mathbf{E} | = \frac{\sigma}{2} |\mathbf{E}|^2
$$

Dabei gilt:

$\sigma$ ist die elektrische Leitfähigkeit (z. B. $56 \times 10^6 \, \text{S/m}$ für Kupfer)

$\mathbf{E}$ ist das elektrische Feld

$$
\mathbf{J} = \sigma \mathbf{E} \quad \text{(Ohmsches Gesetz für Leiter)}
$$

Die Größe $q$ beschreibt die Wärmeerzeugung pro Volumen $(\text{W/m}^3)$ aufgrund des induktiven Joule-Effekts.

Im rotationssymmetrischen Fall ist das elektrische Feld gegeben durch:

$$
\mathbf{E} = -j \omega A_\varphi \quad \text{(laut Formel (6))}
$$

Da wir das Quadrat des Betrags benötigen:

$$
|\mathbf{E}|^2 = |\omega A_\varphi|^2 = \omega^2 |A_\varphi|^2
$$

Daraus ergibt sich die Wärmequelle $q$:

$$
q = \frac{\sigma}{2} \omega^2 |A_\varphi|^2
$$

In [ ]:
### Listing 3 : Berechnung der induktiven Wärmequelle
Ez = -1j*omega*gfu
Jz = sigma*Ez
Jtot = Jz + Jimp
Qe = 1/2*Norm(InnerProduct(Jtot,Conj(Ez)))

Qe = CoefficientFunction(Qe,)

Draw(Qe, mesh)


WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene

In [ ]:
### Listing 4 : FE-Raum für die Temperaturverteilung
V2 = H1(mesh, order = 4, dirichlet = 'outer')
uT,vT = V2.TnT()
gfT = GridFunction(V2)



In [ ]:
### Listing 5 : Bilinearform Temperaturfeld
aT = BilinearForm(V2)
aT += lam*grad(uT)*grad(vT)*dx

fT = LinearForm(V2)
fT += Qe*vT*dx

aT.Assemble()
fT.Assemble()



In [ ]:
### Listing 6 : Geometrie Beispiel
from netgen.geom2d import SplineGeometry
l_air = 0.1
r_wire = 0.001
nx_wire = 10
ny_wire = 3
dxdy_wire = 0.5*r_wire

rA_core = 0.01
rI_core = 0.008
l_core = 0.08
epsZero = 1e-8

geo = SplineGeometry()
geo.AddRectangle(p1=(-l_air/2,epsZero),
                 p2=(l_air/2,l_air/2),
                 bcs=["rotsym","outer","outer","outer"],
                 leftdomain=1,
                 rightdomain=0)
pts = []
for i in range(nx_wire):
    for j in range(ny_wire):
        pts.append([
            i*(2*r_wire+dxdy_wire), 
            rA_core+dxdy_wire+r_wire+j*(2*r_wire+dxdy_wire)
        ])
        geo.AddCircle(
            c=(i*(2*r_wire+dxdy_wire), 
               rA_core+dxdy_wire+r_wire+j*(2*r_wire+dxdy_wire)),
            r=r_wire, bc="inner",
            leftdomain=2,
            rightdomain=1)
pts = np.array(pts)
geo.AddRectangle(p1=(-l_core/2,rI_core),
                 p2=(l_core/2,rA_core),
                 bc="inner",
                 leftdomain=3,
                 rightdomain=1)
geo.SetMaterial (1, "air")
geo.SetMaterial (2, "copper")
geo.SetMaterial (3, "core")
geo.SetDomainMaxH(3,(rA_core-rI_core)/2)

mesh = Mesh(geo.GenerateMesh(maxh=0.0025))

### Listing 7 : Definition der vom Gebiet abhängigen Materialparameter.
omega = 2*pi*50
mu0 = 4*pi*1e-7

sigmaCu = 56e6
sigmaCore = 56e6 #Kupfer
sigma = CoefficientFunction([0, sigmaCu, sigmaCore])
mur = CoefficientFunction([1,1,1])
nu = CoefficientFunction(1/mu0*1/mur)